<a href="https://colab.research.google.com/github/yungsinatra0/Big-Jah/blob/main/ThemisAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Welcome to Themis.AI! This notebook will be used to provide a general summary based on uploaded reports.

## Installing necessary dependencies for dealing with text & pdf

In [1]:
!pip install -q reportlab
!pip install -q patool
!pip install -q PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 656.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 45.8 MB/s eta 0:00:00


## Uploading reports & reading file content

Below you will find 3 ways to upload the reports:


1.   Uploading from local machine - uploading reports stored on your PC/laptop (will need to archive the directories)
2.   Uploading from Google Drive - connecting to your Google Drive account and using the files stored in there
3. Reports already uploaded to Google Colab notebook



### OPTION 1 - Uploading from local machine

Please make sure your reports (pdf files) and directories (folders) are archived together for ease of upload. If you are using Windows, [7-zip](https://www.7-zip.org) or [WinRar](https://www.rarlab.com/download.htm) are excellent and free tools that can be used to do this.

Please refer to [this guide](https://www.howtogeek.com/276972/the-best-file-archiving-program-for-windows/) or [this guide](https://www.wikihow.com/Archive-Folders) for guidance on how to archive the folders.

In [ ]:
# Necessary imports for file handling
import os
from google.colab import files
import re
import patoolib

# Prompt the user to upload files
uploaded_files = files.upload()

# Unpack the archive
archive_name = next(iter(uploaded_files))
current_directory = os.getcwd()
patoolib.extract_archive(archive_name, outdir=current_directory)

# Get parent directory path
directory_path = os.path.join(current_directory, os.path.splitext(archive_name)[0])

# print(directory_path)
print("File extraction completed.")

Saving shoigu.zip to shoigu.zip
patool: Extracting shoigu.zip ...
patool: running /usr/bin/7z x -o/content -- shoigu.zip
patool: ... shoigu.zip extracted to `/content'.
File extraction completed.


### OPTION 2 - Uploading from Google Drive

Please make sure the folder containing the reports is named "Themis.AI" and is located at the root of your Google Drive!

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount("/content/drive")

# Path to the "My Drive" directory
my_drive_path = "/content/drive/My Drive"

# Check if "Themis.AI" folder exists within "My Drive"
themis_folder_path = os.path.join(my_drive_path, "Themis.AI_test")
if os.path.exists(themis_folder_path):
    directory_path = os.path.join(themis_folder_path, os.listdir(themis_folder_path)[0])
else:
    print("Could not find 'Themis.AI' folder in 'My Drive'")

Mounted at /content/drive
Could not find 'Themis.AI' folder in 'My Drive'


In [ ]:
# Unmount drive
drive.flush_and_unmount()
print("All changes made in this colab session should now be visible in Drive.")

### OPTION 3 - Reports already uploaded to Colab

In [ ]:
import os

person_id = input("Please input the person's ID whose reports were uploaded: ")

current_dir = os.getcwd()

directory_path = os.path.join(current_dir, person_id)

Please input the person's ID whose reports were uploaded: shoigu


## Reading file content

In [ ]:
import fitz  # Import the fitz module from PyMuPDF

# Initialization of some variables
documents = {}

# Retrieve the person ID from the directory path
person_id = os.path.basename(directory_path)

# Check if the directory exists
if os.path.exists(directory_path):
    # Traverse through the report type directories
    for root, dirs, files in os.walk(directory_path):
        for directory in dirs:
            report_type = directory.lower().strip()

            # Store the report type if it's not already stored
            if report_type not in documents:
                documents[report_type] = []

            # Traverse through the PDF files
            pdf_files = []
            for file in os.listdir(os.path.join(root, directory)):
                if file.endswith(".pdf"):
                    pdf_files.append(file)

            # Initialize the concatenated text for each report
            report_texts = []

            # Read and concatenate the text from PDF files
            for pdf_file in pdf_files:
                file_path = os.path.join(root, directory, pdf_file)

                # Read the PDF file content using PyMuPDF (fitz)
                pdf_document = fitz.open(file_path)
                pdf_text = [page.get_text().strip() for page in pdf_document]
                report_info = {
                    "file_name": pdf_file,
                    "report_text": " ".join(pdf_text).replace("\n", " "),
                }
                documents[report_type].append(report_info)
                pdf_document.close()


else:
    print("Directory not found.")

print(documents)

{'football': [{'file_name': 'shoigu-12072023-football.pdf', 'report_text': 'Musician Alex Yatsun’s house was shelled by Russian forces, but he has focused the trauma  and apocalyptic feeling into atmospheric tracks that help him get ‘out of reality’  When the Russian invasion of Ukraine began in 2022, Alex Yatsun was living just 30km  from the Russian border in the northernmost part of Kharkiv. “When I woke up that day I  started living in a completely different reality,” he recalls. “There were bombs falling every  hour.”-  Yatsun’s family evacuated but he soon returned north to more dangerous territory to  volunteer at a medical centre. “But my house was hit by shelling,” he recalls. A photograph  on his Instagram shows the aftermath: huge chunks of wall blown out, smashed windows, a  mangled front door. “That was the moment I decided to move closer to central Ukraine.”  Despite the difficult and often harrowing backdrop of the last year, the 24-year-old has  managed to produce a new

# Choosing number of reports to be used for summarization

In [ ]:
# Print all the keys of the "documents" dictionary along with the count of reports
print("Available report types:")
for report_type in list(documents.keys()):
    print(f"{report_type} - {len(documents[report_type])} report(s)")

# Ask the user for input to select report types
chosen_reports = []
chosen_texts = {}
report_types = list(documents.keys())
while True:
    user_input = input(
        "Enter the report types you want to choose (separated by commas), or 'all' for all reports: "
    )
    if user_input.lower() == "all":
        chosen_reports = report_types
        break

    chosen_reports = [report.strip().lower() for report in user_input.split(",")]

    # Validate user input
    invalid_reports = [
        report for report in chosen_reports if report not in report_types
    ]
    if len(invalid_reports) > 0:
        print("Invalid report types:", ", ".join(invalid_reports))
    else:
        break

# Create a new dictionary containing the text of the chosen documents
for report in chosen_reports:
    print(f"\nAvailable reports for '{report}':")
    for index, report_info in enumerate(documents[report], 1):
        print(f"{index}. {report_info['file_name']}")
    while True:
        user_input = input(
            f"Enter the index of the report(s) you want to choose for '{report}' (separated by commas), or 'all' for all reports: "
        )
        if user_input.lower() == "all":
            chosen_texts[report] = [
                report_info["report_text"] for report_info in documents[report]
            ]
            break

        chosen_indices = [
            int(idx.strip())
            for idx in user_input.split(",")
            if 1 <= int(idx.strip()) <= len(documents[report])
        ]
        try:
            chosen_texts[report] = [
                documents[report][idx - 1]["report_text"] for idx in chosen_indices
            ]
            break
        except ValueError:
            print("Invalid input. Please enter valid indices or 'all' for all reports.")
            continue

print(chosen_texts)

Available report types:
football - 1 report(s)
police - 1 report(s)
school - 2 report(s)
Enter the report types you want to choose (separated by commas), or 'all' for all reports: all

Available reports for 'football':
1. shoigu-12072023-football.pdf
Enter the index of the report(s) you want to choose for 'football' (separated by commas), or 'all' for all reports: all

Available reports for 'police':
1. shoigu-10072023-police.pdf
Enter the index of the report(s) you want to choose for 'police' (separated by commas), or 'all' for all reports: all

Available reports for 'school':
1. shoigu-12072023-school.pdf
2. shoigu-10072023-school.pdf
Enter the index of the report(s) you want to choose for 'school' (separated by commas), or 'all' for all reports: all
{'football': ['Musician Alex Yatsun’s house was shelled by Russian forces, but he has focused the trauma  and apocalyptic feeling into atmospheric tracks that help him get ‘out of reality’  When the Russian invasion of Ukraine began in 2

# Abstractive summarization models

**WARNING: SOME MODELS HAVE WORD INPUT LIMITS, PLEASE REFER BELOW TO HOW MANY WORDS CAN BE USED FOR ONE SUMMARIZATION TASK**


*   PRIMERA: 4096 tokens for a 'report cluster' (each document will be 4096 tokens divided by number of documents in the cluster).
*   BRIO: 512 (BART base) or 1024 tokens (PEGASUS base) - depending on model or dataset used, result may vary, but the 512 tokens version is recommended.
*   EfactSum: 512 (BART base) or 1024 tokens (PEGASUS base) - depending on model or dataset used, result may vary, but the 512 tokens version is recommended.
* Unlimiformer: "unlimited input"

In [2]:
!pip install -q transformers torch accelerate datasets sentencepiece evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00


## Load the healthcare report from Google Drive

In [ ]:
!apt install ocrmypdf
!pip install ocrmypdf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript
  icc-profiles-free libgs9 libgs9-common libidn12 libijs-0.35 libimagequant0
  libjbig2dec0 libqpdf28 libraqm0 mailcap mime-support pngquant poppler-data
  python3-bs4 python3-chardet python3-coloredlogs python3-html5lib
  python3-humanfriendly python3-img2pdf python3-lxml python3-olefile
  python3-packaging python3-pdfminer python3-pikepdf python3-pil
  python3-pluggy python3-renderpm python3-reportlab python3-reportlab-accel
  python3-soupsieve python3-tqdm python3-webencodings tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd unpaper
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x ocrmypdf-doc python-watchdog img2pdf poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipaf

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount("/content/drive")

# Path to the "My Drive" directory
my_drive_path = "/content/drive/My Drive"

# Check if "Themis.AI" folder exists within "My Drive"
themis_folder_path = os.path.join(my_drive_path, "SummarisationCalin")
if os.path.exists(themis_folder_path):
    directory_path = os.path.join(themis_folder_path, os.listdir(themis_folder_path)[0])
else:
    print("Could not find 'Themis.AI' folder in 'My Drive'")

report_path = os.path.join(themis_folder_path, "Report")

Mounted at /content/drive


In [ ]:
"""
Demo script using Mupdf OCR.

Extract text of a page and interpret unrecognized characters using Tesseract.
MuPDF codes unrecognizable characters as 0xFFFD = 65533.
Extraction option is "dict", which delivers contiguous text pieces within one
line, that have the same font properties (color, fontsize, etc.). Together with
the language parameter, this helps Tesseract finding the correct character.

The basic approach is to only invoke OCR, if the span text contains
chr(65533). Because Tesseract's response ignores leading spaces and appends
line break characters, some adjustments are made.

--------------
This demo will OCR only text, that is known to be text. This means, it
does not look at parts of a page containing images or text encoded as drawings.
--------------

Dependencies:
PyMuPDF v1.19.0
"""
import os
os.environ["TESSDATA_PREFIX"] = "/usr/share/tesseract-ocr/4.00/tessdata"
import fitz
import time

mat = fitz.Matrix(5, 5)  # high resolution matrix
ocr_time = 0
pix_time = 0
INVALID_UNICODE = chr(0xFFFD)  # the "Invalid Unicode" character


def get_tessocr(page, bbox):
    """Return OCR-ed span text using Tesseract.

    Args:
        page: fitz.Page
        bbox: fitz.Rect or its tuple
    Returns:
        The OCR-ed text of the bbox.
    """
    global ocr_time, pix_time, tess, mat
    # Step 1: Make a high-resolution image of the bbox.
    t0 = time.perf_counter()
    pix = page.get_pixmap(
        matrix=mat,
        clip=bbox,
    )
    t1 = time.perf_counter()
    ocrpdf = fitz.open("pdf", pix.pdfocr_tobytes())
    ocrpage = ocrpdf[0]
    text = ocrpage.get_text()
    if text.endswith("\n"):
        text = text[:-1]
    t2 = time.perf_counter()
    ocr_time += t2 - t1
    pix_time += t1 - t0
    return text

pdf_file = "HSIB Final Report.pdf"
file_path = os.path.join(report_path, pdf_file)
chosen_pages = [34, 35, 36]
page_text_list = []

doc = fitz.open(file_path)
ocr_count = 0
for page in doc:
    if page.number in chosen_pages:
      page_text = ""
      blocks = page.get_text("dict", flags=0)["blocks"]
      for b in blocks:
          for l in b["lines"]:
              for s in l["spans"]:
                  text = s["text"]
                  if INVALID_UNICODE in text:  # invalid characters encountered!
                      # invoke OCR
                      ocr_count += 1
                      # print("before: '%s'" % text)
                      text1 = text.lstrip()
                      sb = " " * (len(text) - len(text1))  # leading spaces
                      text1 = text.rstrip()
                      sa = " " * (len(text) - len(text1))  # trailing spaces
                      new_text = sb + get_tessocr(page, s["bbox"]) + sa
                      # print(" after: '%s'" % new_text)
                      page_text += new_text
                  else:
                      page_text += text

      page_text_list.append(page_text)


page_text_list[0] = page_text_list[0] + page_text_list[1]
del page_text_list[1]


# Print accumulated text for each page
for page_num, page_text in enumerate(page_text_list):
    print(f"Page {chosen_pages[page_num]} Text:")
    print(page_text)
    print("-------------------------")

Page 34 Text:
 35  Section 6. HSIB Findings and Safety Recommendations  6.1 Findings 1. The Mother booked for maternity care, having returned to the UK from overseas. She had no documentation of the care she had received. 2. The Mothers estimated date of birth from overseas place her at 37+4 weeks. After clinical review the ongoing intention was to utilise this date, this was not clearly documented and late gestation EDD from the Trust USS was used. 3. As an EDD from a late USS was used in care planning. This placed the Mother’s pregnancy two weeks and four days earlier than the correct gestation.  4. The Mother had delivered by prior CS and requested this mode of delivery again. Latent phase of labour occurred prior to the planned CS date. 5. The Mother presented in the latent phase of labour at 36+2 weeks (Trust USS), 38+6 (overseas USS).  From admission the gestation was communicated as 36+2 weeks, this may have influenced the clinical decisions in not proceeding with a CS at the Mo

## PRIMERA Model

Code taken from https://github.com/allenai/PRIMER

Notes on usage:

- Model is pre-trained on a news-based dataset, so it may hallucinate when used in a different task/context.
- Model can handle ~2000 words at once.
- Model does not work well if the input length is too low (less than 300-500 words) or if too high (more than ~2500 words)
- Model can handle multiple documents on the same topic, but pre-processing beforehand is necessary.
- While the model can handle bigger inputs & multiple documents, the summarization process can take longer in terms of time (can be over 10 minutes!)

Evaluation results:

| Dataset | ROUGUE-1 | ROUGUE-2 | ROUGUE-L|
| --- | --- | --- | --- |
| Multi-news | 46.93 | 18.86 | 26.78 |
| CNNDM | 12.37 | 0.96 | 8.12 |

For more details on the metric, check out:

    ROUGE-n recall=40% means that 40% of the n-grams in the reference summary are also present in the generated summary.
    ROUGE-n precision=40% means that 40% of the n-grams in the generated summary are also present in the reference summary.
    ROUGE-n F1-score=40% is more difficult to interpret, like any F1-score.

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/introduction-to-text-summarization-with-rouge-scores-84140c64b471

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460


### Importing modules & initializing variables used for the model

In [ ]:
# Importing necessary modules
from transformers import (
    AutoTokenizer,
    LEDForConditionalGeneration,
    LEDConfig,
)
from datasets import load_dataset
import torch
import evaluate


# Initializing variables
TOKENIZER = AutoTokenizer.from_pretrained("allenai/PRIMERA-multinews")
CONFIG = LEDConfig.from_pretrained("allenai/PRIMERA-multinews")
MODEL = LEDForConditionalGeneration.from_pretrained("allenai/PRIMERA-multinews", config=CONFIG)
# MODEL.gradient_checkpointing_enable()
PAD_TOKEN_ID = TOKENIZER.pad_token_id
DOCSEP_TOKEN_ID = TOKENIZER.convert_tokens_to_ids("<doc-sep>")

### Pre-processing the document(s) specifically for that model

In [ ]:
print(chosen_texts)

# Create a new list containing the concatenated texts of the chosen documents
concatenated_texts_list = []
for report_texts in chosen_texts.values():
    concatenated_texts_list.append("|||||".join(report_texts))

print(concatenated_texts_list)

print(len(concatenated_texts_list))

{'football': ['Musician Alex Yatsun’s house was shelled by Russian forces, but he has focused the trauma  and apocalyptic feeling into atmospheric tracks that help him get ‘out of reality’  When the Russian invasion of Ukraine began in 2022, Alex Yatsun was living just 30km  from the Russian border in the northernmost part of Kharkiv. “When I woke up that day I  started living in a completely different reality,” he recalls. “There were bombs falling every  hour.”-  Yatsun’s family evacuated but he soon returned north to more dangerous territory to  volunteer at a medical centre. “But my house was hit by shelling,” he recalls. A photograph  on his Instagram shows the aftermath: huge chunks of wall blown out, smashed windows, a  mangled front door. “That was the moment I decided to move closer to central Ukraine.”  Despite the difficult and often harrowing backdrop of the last year, the 24-year-old has  managed to produce a new 21-track compilation album as DJ Sacred, entitled Dungeon Ra

### Use Transformer pipelines to summarize text  (high level & easier to use)


It is possible to change the following parameters within the pipe() call to get a different summarization result:

* Max_length: Set the maximum length of the input + output sequence (can use max_new_tokens to set just the output sequence length)
* Min_length: Set the minimum length of the output sequence (recommended to set max_length as well, and it should be longer than min_length)
* Temperature: Temperature affects how “random” the model’s output is. Lower value = lower "randomness".
* Top_p: Arrange given tokens by probability, select the fewest needed to reach a cumulative probability of at least p, and then sample from them (need to have do_sample set to 'True').

For more details on temperature, top_p: https://peterchng.com/blog/2023/05/02/token-selection-strategies-top-k-top-p-and-temperature/

In [ ]:
# Use a pipeline as a high-level helper
import torch
from transformers import pipeline

pipe = pipeline(
    task = "text2text-generation",
    model = MODEL,
    tokenizer = TOKENIZER,
    # torch_dtype=torch.bfloat16,
    # device="auto"
)

# Use model
result = pipe(
    concatenated_texts_list,
    use_cache = True,
    # min_length = 256,
    num_beams = 5,
    # max_length = 1024,
    pad_token_id = TOKENIZER.pad_token_id,
    bos_token_id = TOKENIZER.bos_token_id,
    eos_token_id = TOKENIZER.eos_token_id,
    # do_sample=True, # Only necessary to enable if want to use temperature or top_p parameters
    # temperature=0.1, # Will control
    # top_p=0.3
    )

print(result)

[{'generated_text': '– When the Russian invasion of Ukraine began in 2022, Alex Yatsun was living just 30km from the Russian border in the northernmost part of Kharkiv. "When I woke up that day I started living in a completely different reality," he recalls. "There were bombs falling every hour." His family evacuated, but he returned to volunteer at a medical center. "But my house was hit by shelling," he says. "That was the moment I decided to move closer to central Ukraine." Despite the difficult and often harrowing backdrop of the last year, the 24-year-old has managed to produce a new 21-track compilation album as DJ Sacred, entitled Dungeon Rap: The Evolution. It\'s a follow-up to 2019\'s, Dungeon rap: the Introduction. The album is a new hybrid style of hip-hop created by Yatsin, the Independent reports.'}, {'generated_text': "– A senior Russian draft officer and former submarine commander accused by Ukraine of deadly strikes on its territory has been shot dead while jogging in t

## BRIO Model

Code taken from https://github.com/yixinL7/BRIO

Notes on usage:

- Model is pre-trained on a news-based dataset, so it may hallucinate when used in a different task/context.
- Model can handle ~350-750 words at once (depending on the base model used).
- Model only handly a single documents at a time.
- Model processing time is usually quite fast, depending on chosen parameters (1-3 minutes per document).

Evaluation results:

| Dataset | ROUGUE-1 | ROUGUE-2 | ROUGUE-L|
| --- | --- | --- | --- |
| Multi-news | 26.4 | 9.9 | 16.2 |
| CNNDM | 46.8 | 22.4 | 31.5 |

For more details, check out:

    ROUGE-n recall=40% means that 40% of the n-grams in the reference summary are also present in the generated summary.
    ROUGE-n precision=40% means that 40% of the n-grams in the generated summary are also present in the reference summary.
    ROUGE-n F1-score=40% is more difficult to interpret, like any F1-score.

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/introduction-to-text-summarization-with-rouge-scores-84140c64b471

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460


### Load the model using Hugging Face library

BRIO comes with 2 options of pre-trained weights to be loaded:
- PEGASUS-base: Trained on XSUM dataset (https://huggingface.co/datasets/xsum), has a maximum input length of 1024 tokens. Output will be a single sentence.
- BART-base: Trained on CNNDM dataset (https://huggingface.co/datasets/cnn_dailymail), has a maximum input length of 512 tokens. Output will be one or multiple sentences (depending on input length).

To choose the model, please change the value of IS_CNNDM to either:
- True, for BART-base
- False, for PEGASUS-base

In [ ]:
from transformers import BartTokenizer, PegasusTokenizer
from transformers import BartForConditionalGeneration, PegasusForConditionalGeneration

IS_CNNDM = True # whether to use CNNDM dataset (BART-base) or XSum dataset (PEGASUS-base)
LOWER = False

# Load our model checkpoints
if IS_CNNDM:
    model = BartForConditionalGeneration.from_pretrained('Yale-LILY/brio-cnndm-uncased')
    tokenizer = BartTokenizer.from_pretrained('Yale-LILY/brio-cnndm-uncased')
else:
    model = PegasusForConditionalGeneration.from_pretrained('Yale-LILY/brio-xsum-cased')
    tokenizer = PegasusTokenizer.from_pretrained('Yale-LILY/brio-xsum-cased')

max_length = 1024 if IS_CNNDM else 512

### Use the model to summarize text

It is possible to change the following parameters within the generate() call to get a different summarization result:

* Max_length: Set the maximum length of the input + output sequence (can use max_new_tokens to set just the output sequence length)
* Min_length: Set the minimum length of the output sequence (recommended to set max_length as well, and it should be longer than min_length)
* Temperature: Temperature affects how “random” the model’s output is. Lower value = lower "randomness".
* Top_p: Arrange given tokens by probability, select the fewest needed to reach a cumulative probability of at least p, and then sample from them (need to have do_sample set to 'True').

For more details on temperature, top_p: https://peterchng.com/blog/2023/05/02/token-selection-strategies-top-k-top-p-and-temperature/

In [ ]:
# Initialize an empty list to store all the resulted summaries
result = []

# Loop through each report type in the chosen_texts dictionary
for report_type, documents in chosen_texts.items():
    # Loop through each document in the list for the current report type
    for document in documents:
        if LOWER:
            article = document.lower()
        else:
            article = document

        # Tokenize the document and generate the summary
        inputs = tokenizer([article], max_length=max_length, return_tensors="pt", truncation=True)
        summary_ids = model.generate(
            inputs["input_ids"],
            # max_length=1024,
            # min_length=128,
            # do_sample=True,
            # temperature=0.1,
            # top_p=0.3
            )
        summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


        # Append the summary to the list of resulted_summaries
        result.append(summary)

# Print all the summaries
for idx, summary in enumerate(result, 1):
    print(f"Summary {idx}: {summary}")

## EFactsum

Code taken from https://github.com/tanay2001/efactsum

Notes on usage:

- Model is pre-trained on a news-based dataset, so it may hallucinate when used in a different task/context.
- Model can handle ~350-750 words at once (depending on the base model used).
- Model only handly a single documents at a time.
- Model processing time is usually quite fast, depending on chosen parameters (1-3 minutes per document).

Evaluation results:

| Dataset | ROUGUE-1 | ROUGUE-2 | ROUGUE-L|
| --- | --- | --- | --- |
| Multi-news | 25 | 8.5 | 15 |
| CNNDM | 46 | 23.1 | 30.6 |

For more details, check out:

    ROUGE-n recall=40% means that 40% of the n-grams in the reference summary are also present in the generated summary.
    ROUGE-n precision=40% means that 40% of the n-grams in the generated summary are also present in the reference summary.
    ROUGE-n F1-score=40% is more difficult to interpret, like any F1-score.

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/introduction-to-text-summarization-with-rouge-scores-84140c64b471

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460


### Load the model using Hugging Face library

EFactSum comes with 2 options of pre-trained weights to be loaded:

- PEGASUS-base: Trained on XSUM dataset (https://huggingface.co/datasets/xsum), has a maximum input length of 1024 tokens. Output will be a single sentence.
- BART-base: Trained on CNNDM dataset (https://huggingface.co/datasets/cnn_dailymail), has a maximum input length of 512 tokens. Output will be one or multiple sentences (depending on input length).

To choose the model, please change the value of IS_CNNDM to either:

- True, for BART-base
- False, for PEGASUS-base

In [ ]:
from transformers import BartTokenizer, PegasusTokenizer
from transformers import BartForConditionalGeneration, PegasusForConditionalGeneration

IS_CNNDM = True
max_length = 1024 if IS_CNNDM else 512

if IS_CNNDM:
    model = BartForConditionalGeneration.from_pretrained('tanay/efactsum-bart-cnndm')
    tokenizer = BartTokenizer.from_pretrained('tanay/efactsum-bart-cnndm')
else:
    model = PegasusForConditionalGeneration.from_pretrained('tanay/efactsum-pegasus-xsum')
    tokenizer = PegasusTokenizer.from_pretrained('tanay/efactsum-pegasus-xsum')

### Use the model to summarize text

It is possible to change the following parameters within the generate() call to get a different summarization result:

* Max_length: Set the maximum length of the input + output sequence (can use max_new_tokens to set just the output sequence length)
* Min_length: Set the minimum length of the output sequence (recommended to set max_length as well, and it should be longer than min_length)
* Temperature: Temperature affects how “random” the model’s output is. Lower value = lower "randomness".
* Top_p: Arrange given tokens by probability, select the fewest needed to reach a cumulative probability of at least p, and then sample from them (need to have do_sample set to 'True').

For more details on temperature, top_p: https://peterchng.com/blog/2023/05/02/token-selection-strategies-top-k-top-p-and-temperature/

In [ ]:
# Initialize an empty list to store all the resulted summaries
result = []

# Loop through each report type in the chosen_texts dictionary
for report_type, documents in chosen_texts.items():
    # Loop through each document in the list for the current report type
    for document in documents:
        if LOWER:
            article = document.lower()
        else:
            article = document

        # Tokenize the document and generate the summary
        inputs = tokenizer([article], max_length=max_length, return_tensors="pt", truncation=True)
        summary_ids = model.generate(
            inputs["input_ids"]
            # max_length=1024,
            # min_length=128,
            # do_sample=True,
            # temperature=0.1,
            # top_p=0.3
            )
        summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


        # Append the summary to the list of resulted_summaries
        result.append(summary)

# Print all the summaries
for idx, summary in enumerate(result, 1):
    print(f"Summary {idx}: {summary}")

## Unlimiformer model



Code taken from https://github.com/abertsch72/unlimiformer

Notes on usage:

- Model is pre-trained on a US government report dataset, so it may hallucinate when used in a different task/context.
- Model can handle an unlimited input context (tested up to ~6000 words).
- Model only handly a single documents at a time.
- Model processing time is usually quite slow (can take up to 15 minutes per document).
- Model is trained on long documents mostly, so may have a poor performance with short documents (shorter than ~1000 words).

Evaluation results:

| Dataset | ROUGUE-1 | ROUGUE-2 | ROUGUE-L|
| --- | --- | --- | --- |
| Multi-news | 38.5 | 11.7 | 18.4 |
| CNNDM | 28.4 | 11.5 | 17.4 |

For more details, check out:

    ROUGE-n recall=40% means that 40% of the n-grams in the reference summary are also present in the generated summary.
    ROUGE-n precision=40% means that 40% of the n-grams in the generated summary are also present in the reference summary.
    ROUGE-n F1-score=40% is more difficult to interpret, like any F1-score.

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/introduction-to-text-summarization-with-rouge-scores-84140c64b471

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460


### Install & import necessary modules

In [3]:
!git clone https://github.com/abertsch72/unlimiformer.git
!pip install -q -r unlimiformer/requirements.txt
!pip install -q faiss-cpu
%cd unlimiformer/src

Cloning into 'unlimiformer'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (307/307), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 449 (delta 237), reused 265 (delta 210), pack-reused 142
Receiving objects: 100% (449/449), 301.72 KiB | 1.77 MiB/s, done.
Resolving deltas: 100% (302/302), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [4]:
from unlimiformer import Unlimiformer
from random_training_unlimiformer import RandomTrainingUnlimiformer
from usage import UnlimiformerArguments, training_addin

from transformers import BartForConditionalGeneration, AutoTokenizer
from datasets import load_dataset
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# example using govreport
modelname = "abertsch/unlimiformer-bart-govreport-alternating"

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained(modelname)

defaults = UnlimiformerArguments()
unlimiformer_kwargs = {
            'layer_begin': defaults.layer_begin,
            'layer_end': defaults.layer_end,
            'unlimiformer_head_num': defaults.unlimiformer_head_num,
            'exclude_attention': defaults.unlimiformer_exclude,
            'chunk_overlap': defaults.unlimiformer_chunk_overlap,
            'model_encoder_max_len': defaults.unlimiformer_chunk_size,
            'verbose': defaults.unlimiformer_verbose, 'tokenizer': tokenizer,
            'unlimiformer_training': defaults.unlimiformer_training,
            'use_datastore': defaults.use_datastore,
            'flat_index': defaults.flat_index,
            'test_datastore': defaults.test_datastore,
            'reconstruct_embeddings': defaults.reconstruct_embeddings,
            'gpu_datastore': defaults.gpu_datastore,
            'gpu_index': defaults.gpu_index
}

model.to(device)

model = Unlimiformer.convert_model(model, **unlimiformer_kwargs)
model.eval()
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

### Summarize text

In [ ]:
# print(f"INPUT LENGTH (tokens): {example['input_ids'].shape[-1]}")

example = tokenizer(example_input, truncation=False, return_tensors="pt")
example.to(device)

unlimiformer_out = tokenizer.batch_decode(model.generate(**example, max_length=512), ignore_special_tokens=True)[0]
print(unlimiformer_out)

KeyboardInterrupt: ignored

# Extractive summarization methods

## Memsum

Code taken from https://github.com/nianlonggu/MemSum

Notes on usage:

- Model is pre-trained on a US government report dataset, so it may hallucinate when used in a different task/context.
- Model can handle ~700 token input context (~500 words).
- Model only handly a single documents at a time.
- Model processing time is quite fast (~1 min per document, depending on size).

Evaluation results:

| Dataset | ROUGUE-1 | ROUGUE-2 | ROUGUE-L|
| --- | --- | --- | --- |
| Multi-news | 4.5 | 2.2 | 4.5 |
| CNNDM | 17.5 | 4 | 17.4 |
| GovReport| 60 | 29.3 | 57.2 |

For more details, check out:

    ROUGE-n recall=40% means that 40% of the n-grams in the reference summary are also present in the generated summary.
    ROUGE-n precision=40% means that 40% of the n-grams in the generated summary are also present in the reference summary.
    ROUGE-n F1-score=40% is more difficult to interpret, like any F1-score.

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/introduction-to-text-summarization-with-rouge-scores-84140c64b471

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460


### Install & import the necessary modules

In [4]:
!git clone https://github.com/nianlonggu/MemSum.git
!pip install -q torch torchvision torchaudio
!pip install -q -r MemSum/requirements.txt

fatal: destination path 'MemSum' already exists and is not an empty directory.


### Tokenize input text into separate sentences.

Necessary to do so as the model only reads a list of sentences, not the sentences concatenated together in one string!

In [ ]:
import nltk
nltk.download('punkt')  # Download the necessary data for sentence tokenization

from nltk.tokenize import sent_tokenize

# Create list from dictionary first

doc_list = []

for key, value_list in chosen_texts.items():
        doc_list.extend(value_list)

# Tokenize each document into sentences and create a list of lists
list_of_lists = []
for doc in doc_list:
    sentences = sent_tokenize(doc)
    list_of_lists.append(sentences)

# Print the resulting list of sentences
print(list_of_lists[2])

['A senior Russian draft officer and former submarine commander accused by Ukraine of  deadly strikes on its territory has been shot dead while jogging in the southern Russian city of  Krasnodar.', 'Stanislav Rzhitsky, 42, was killed on Monday by an unidentified gunman during a morning  run in a park near the Olimp sports centre, local police said.', 'Russian FSB security services said on Tuesday that a 64-year-old man was arrested on  suspicion of carrying out the attack.', 'At the time of his death, Rzhitsky was serving as the deputy head of the Krasnodar city  administration’s mobilisation.', 'According to the Russian daily newspaper Kommersant, Rzhitsky was previously the  commander of the Krasnodar submarine, named after the city, in the Russian navy.', 'The Ukrainian army said in a Telegram post on Tuesday that Rzhitsky was in command of a  submarine that carried out a deadly missile attack on the Ukrainian city of Vinnytsia in July  2022, killing 23 civilians.', 'Rzhitsky’s fath

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Download & import model weights

In [6]:
%cd MemSum
from huggingface_hub import snapshot_download
from src.summarizer import MemSum
from tqdm import tqdm
import json
import numpy as np

## download the pretrained glove word embedding (200 dimension)
snapshot_download('nianlong/memsum-word-embedding', local_dir = "model/word_embedding" )

## download model checkpoint on the Gov-Report dataset
snapshot_download('nianlong/memsum-gov-report', local_dir = "model/memsum-gov-report" )

memsum_gov_report = MemSum(  "model/memsum-gov-report/model.pt",
                  "model/word_embedding/vocabulary_200dim.pkl",
                  gpu = 0 ,  max_doc_len = 500  )

/content/MemSum


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

### Use model to summarize input text

In [ ]:
result = []

for text in list_of_lists:
  extracted_summary = memsum_gov_report.extract([text],
                                    p_stop_thres = 0.6,
                                    max_extracted_sentences_per_document = 7
                                    )[0]
  result.append(' '.join(extracted_summary))

print(result)


["UK defence minister: 'people want to see a bit of  gratitude' from Ukraine for weapon supplies  Dan Sabbagh is in Vilnius for the Guardian and reports these words from Ben Wallace:  The British defence secretary suggested Ukraine needed to put more emphasis on  saying thank you for western help when he was asked about President Volodymyr  Zelenskiy’s complaints on Tuesday that the country had not been issued a firm  timetable or set of conditions for joining Nato. There was an  acceptance that “Ukraine belongs at Nato” and that amounted to an effective invitation for  membership whenever the conflict died down. Wallace revealed that he had travelled to Ukraine last year to be presented with a shopping  list of weapons. “I told them that last year, when I  drove 11 hours to be given a list.”  But he said he understood Zelenskiy was speaking to his own public and that despite his  complaint on Tuesday, the final summit deal was a good one for Ukraine. “Whether we like it or not, people

### Evaluate using ROUGE

In [27]:
import os
import subprocess
import wget

for dataset_name in [ "gov-report"]:
    print(dataset_name)
    os.makedirs( "data/"+dataset_name, exist_ok=True )

    ## dataset is stored at huggingface hub
    train_dataset_path = f"https://huggingface.co/datasets/nianlong/long-doc-extractive-summarization-{dataset_name}/resolve/main/train.jsonl"
    val_dataset_path = f"https://huggingface.co/datasets/nianlong/long-doc-extractive-summarization-{dataset_name}/resolve/main/val.jsonl"
    test_dataset_path = f"https://huggingface.co/datasets/nianlong/long-doc-extractive-summarization-{dataset_name}/resolve/main/test.jsonl"

    wget.download( train_dataset_path, out = "data/"+dataset_name )
    wget.download( val_dataset_path, out = "data/"+dataset_name )
    wget.download( test_dataset_path, out = "data/"+dataset_name )

test_corpus_gov_report = [ json.loads(line) for line in open("data/gov-report/test.jsonl") ]


gov-report


In [26]:
from rouge_score import rouge_scorer

rouge_cal = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeLsum'], use_stemmer=True)

def evaluate( model, corpus, p_stop, max_extracted_sentences, rouge_cal ):
    scores = []
    for data in tqdm(corpus):
        gold_summary = data["summary"]
        extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]

        score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
        scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure, score["rougeLsum"].fmeasure ] )

    return np.asarray(scores).mean(axis = 0)

In [30]:
from datasets import load_dataset
import random

# dataset=load_dataset('multi_news')
data_idx = random.choices(range(len(test_corpus_gov_report)),k=20)
dataset_small = [test_corpus_gov_report[i] for i in data_idx]
# dataset_small = test_corpus_gov_report.select(data_idx)
evaluate(memsum_gov_report, dataset_small, 0.6, 22, rouge_cal)

100%|██████████| 20/20 [03:23<00:00, 10.17s/it]


array([0.60083427, 0.29311724, 0.57216964])

## BERTSum

Code taken from https://github.com/dmmiller612/bert-extractive-summarizer and https://chriskhanhtran.github.io/posts/extractive-summarization-with-bert/

Notes on usage:

- Model does not specify the maximum input lenght it can handle. It is assume it's around ~500 tokens (~350 words).
- Model only handly a single documents at a time.
- Model processing time is quite fast (~1 min per document, depending on size).

Evaluation results:

| Dataset | ROUGUE-1 | ROUGUE-2 | ROUGUE-L|
| --- | --- | --- | --- |
| Multi-news |  |  |  |
| CNNDM |  |  |  |
| GovReport| |  |  |

For more details, check out:

    ROUGE-n recall=40% means that 40% of the n-grams in the reference summary are also present in the generated summary.
    ROUGE-n precision=40% means that 40% of the n-grams in the generated summary are also present in the reference summary.
    ROUGE-n F1-score=40% is more difficult to interpret, like any F1-score.

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/introduction-to-text-summarization-with-rouge-scores-84140c64b471

https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460


In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
from summarizer import Summarizer

result = []

model = Summarizer()

for report_type, documents in chosen_texts.items():
  for doc in documents:
    text = model(doc, ratio=0.4)
    result.append(text)

print(result)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


['Musician Alex Yatsun’s house was shelled by Russian forces, but he has focused the trauma  and apocalyptic feeling into atmospheric tracks that help him get ‘out of reality’  When the Russian invasion of Ukraine began in 2022, Alex Yatsun was living just 30km  from the Russian border in the northernmost part of Kharkiv. “ Yatsun’s family evacuated but he soon returned north to more dangerous territory to  volunteer at a medical centre. “ Dungeon rap is a new hybrid style of hip-hop created by Yatsun. DJ Armok is more reliant on bass and more  heavily connected to Memphis rap, whereas Pillbox is more about the ethereal and  transcendental – like a sublime melancholic feeling.” The result is moody and atmospheric stuff. He’s assembled them in mega packs, containing thousands of samples, that  can be used as the basis for other people to make their own mutated twist on the genre. There is also a feeling of inescapable darkness to the music. It corresponds to my life and my beliefs that 

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## Sbert

Code taken from https://github.com/dmmiller612/bert-extractive-summarizer and https://chriskhanhtran.github.io/posts/extractive-summarization-with-bert/

Notes on usage:

- Model does not specify the maximum input lenght it can handle. It is assume it's around ~500 tokens (~350 words).
- Model only handly a single documents at a time.
- Model processing time is quite fast (~1 min per document, depending on size).

In [ ]:
!pip install -U -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.2 MB/s eta 0:00:00


In [ ]:
from summarizer.sbert import SBertSummarizer

model = SBertSummarizer('paraphrase-MiniLM-L6-v2')

for report_type, documents in chosen_texts.items():
  for doc in documents:
    text = model(doc, ratio=0.4)
    result.append(text)

print(result)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


['Musician Alex Yatsun’s house was shelled by Russian forces, but he has focused the trauma  and apocalyptic feeling into atmospheric tracks that help him get ‘out of reality’  When the Russian invasion of Ukraine began in 2022, Alex Yatsun was living just 30km  from the Russian border in the northernmost part of Kharkiv. “ Yatsun’s family evacuated but he soon returned north to more dangerous territory to  volunteer at a medical centre. “ Dungeon rap is a new hybrid style of hip-hop created by Yatsun. DJ Armok is more reliant on bass and more  heavily connected to Memphis rap, whereas Pillbox is more about the ethereal and  transcendental – like a sublime melancholic feeling.” The result is moody and atmospheric stuff. He’s assembled them in mega packs, containing thousands of samples, that  can be used as the basis for other people to make their own mutated twist on the genre. There is also a feeling of inescapable darkness to the music. It corresponds to my life and my beliefs that 

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## TransformerSum (RoBERTa or Longformer) - not working yet

https://github.com/HHousen/TransformerSum

In [ ]:
!git clone https://github.com/HHousen/transformersum.git
%cd transformersum

Cloning into 'transformersum'...
remote: Enumerating objects: 1424, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 1424 (delta 99), reused 166 (delta 91), pack-reused 1232
Receiving objects: 100% (1424/1424), 11.98 MiB | 10.40 MiB/s, done.
Resolving deltas: 100% (883/883), done.
/content/transformersum


In [ ]:
!pip install -q pytorch_lightning==1.6.5 transformers==4.* torch_optimizer==0.3.* wandb==0.14.* rouge-score==0.1.* packaging datasets==2.* gradio==3.* torch==2.0.* scikit-learn==1.2.* tensorboard spacy sphinx pyarrow pre-commit==3.2.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:00
ERROR: Cannot install protobuf==3.20.3 and pytorch-lightning==1.6.5 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
!mkdir models
!gdown 1xlBJTO1LF5gIfDNvG33q8wVmvUB4jXYx
!mv epoch=3.ckpt models/epoch=3.ckpt

mkdir: cannot create directory ‘models’: File exists
Downloading...
From: https://drive.google.com/uc?id=1xlBJTO1LF5gIfDNvG33q8wVmvUB4jXYx
To: /content/transformersum/epoch=3.ckpt
100% 1.49G/1.49G [00:14<00:00, 103MB/s] 


In [ ]:
import sys
sys.path.append('/content/transformersum/src')

from extractive import ExtractiveSummarizer
model = ExtractiveSummarizer.load_from_checkpoint("models/epoch=3.ckpt")

result = []

for report_type, documents in chosen_texts.items():
  for doc in documents:
    summary = model.predict(doc, num_summary_sentences=5)
    result.append(summary)

print(result)


Token indices sequence length is longer than the specified maximum sequence length for this model (1454 > 512). Running this sequence through the model will result in indexing errors


['Musician Alex Yatsun ’s house was shelled by Russian forces , but he has focused the trauma   and apocalyptic feeling into atmospheric tracks that help him get ‘ out of reality ’   When the Russian invasion of Ukraine began in 2022 , Alex Yatsun was living just 30 km   from the Russian border in the northernmost part of Kharkiv “.   Despite the difficult and often harrowing backdrop of the last year , the 24 - year - old has   managed to produce a new 21 - track compilation album as DJ Sacred , entitled Dungeon Rap :   the Evolution. It ’s a follow - up to 2019 ’s , Dungeon Rap : the Introduction.   Dungeon rap is a new hybrid style of hip - hop created by Yatsun.   Yatsun ’s compilations feature a variety of his musical aliases , such as DJ Bishop , DJ Armok   and Pillbox.', 'A senior Russian draft officer and former submarine commander accused by Ukraine of   deadly strikes on its territory has been shot dead while jogging in the southern Russian city of   Krasnodar.   Stanislav Rz

## HISum - training in process

https://github.com/MySong7NLPer/HISum

In [ ]:
!gdown 1PxMHpDSvP1OJfj1et4ToklevQzcPr-HQ
!git clone https://github.com/maszhongming/MatchSum.git

Downloading...
From: https://drive.google.com/uc?id=1PxMHpDSvP1OJfj1et4ToklevQzcPr-HQ
To: /content/MatchSum_cnndm_model.zip
100% 855M/855M [00:09<00:00, 87.8MB/s]
Cloning into 'MatchSum'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 65 (delta 16), reused 14 (delta 14), pack-reused 41
Receiving objects: 100% (65/65), 77.84 KiB | 2.43 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
!unzip MatchSum_cnndm_model.zip -d MatchSum

Archive:  MatchSum_cnndm_model.zip
  inflating: MatchSum/MatchSum_cnndm_bert.ckpt  
  inflating: MatchSum/MatchSum_cnndm_roberta.ckpt  


In [ ]:
!pip install -q torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00


In [ ]:
# Create list from dictionary first

doc_list = []

for key, value_list in chosen_texts.items():
        doc_list.extend(value_list)

print(doc_list)

['Musician Alex Yatsun’s house was shelled by Russian forces, but he has focused the trauma  and apocalyptic feeling into atmospheric tracks that help him get ‘out of reality’  When the Russian invasion of Ukraine began in 2022, Alex Yatsun was living just 30km  from the Russian border in the northernmost part of Kharkiv. “When I woke up that day I  started living in a completely different reality,” he recalls. “There were bombs falling every  hour.”-  Yatsun’s family evacuated but he soon returned north to more dangerous territory to  volunteer at a medical centre. “But my house was hit by shelling,” he recalls. A photograph  on his Instagram shows the aftermath: huge chunks of wall blown out, smashed windows, a  mangled front door. “That was the moment I decided to move closer to central Ukraine.”  Despite the difficult and often harrowing backdrop of the last year, the 24-year-old has  managed to produce a new 21-track compilation album as DJ Sacred, entitled Dungeon Rap:  the Evolu

In [ ]:
print(transformers.__version__)

import torch;
torch.__version__

4.31.0


'2.0.1+cu118'

In [ ]:
%cd MatchSum
import torch
import transformers

torch.load('MatchSum_cnndm_roberta.ckpt')

[Errno 2] No such file or directory: 'MatchSum'
/content/MatchSum


ModuleNotFoundError: ignored

# Exporting summary to pdf and downloading the file.

## Creating a pdf file out of the summary result

In [ ]:
from reportlab.pdfgen.canvas import Canvas
import textwrap
from datetime import date
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch

# Create summary PDF file (if it doesn't exist, it will be automatically created)
canvas = Canvas("summary.pdf", pagesize=A4)

# Get A4 sizes
width, length = A4
top_indent = length - inch
left_indent = inch
right_indent = width - inch

# Set the title font, size, and alignment
canvas.setFont("Helvetica-Bold", 16)
canvas.drawCentredString((width * 0.5), (length - 40), "Themis.AI Summary")

# Add person's ID, today's date, and report types chosen
canvas.setFont("Times-Roman", 12)
canvas.drawString(left_indent, top_indent + 10, "Person ID: " + person_id)
canvas.drawString(left_indent, top_indent - 4, "Date: " + str(date.today()))
canvas.drawString(
    left_indent, top_indent - 18, "Report Types: " + ", ".join(chosen_reports)
)

# Set the text position and font
text_x = left_indent
text_y = top_indent - 42
canvas.setFont("Times-Roman", 12)

# Loop through the results
for result_item in result:
    if isinstance(result_item, dict):  # Handle dictionary output
        # Get the generated text from the dictionary
        text = result_item.get("generated_text", "")

        # Wrap the text and draw it on the canvas
        wrapped_text = textwrap.wrap(text, width=80)
        for line in wrapped_text:
            canvas.drawString(text_x, text_y, line)
            text_y -= 14
            if text_y < 14:
                canvas.showPage()
                text_y = top_indent - 28  # Adjust to the next page starting position

    elif isinstance(result_item, str):  # Handle list output (simple string)
        # Wrap the text and draw it on the canvas
        wrapped_text = textwrap.wrap(result_item, width=80)
        for line in wrapped_text:
            canvas.drawString(text_x, text_y, line)
            text_y -= 14
            if text_y < 14:
                canvas.showPage()
                text_y = top_indent - 28  # Adjust to the next page starting position

# Add the text to the PDF file & close it
canvas.save()


## Download the pdf file

In [ ]:
from google.colab import files

files.download("summary.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>